In [ ]:
import tkinter as tk
from tkinter import filedialog
import os
import subprocess

def ADD_TITLE(frame,title):
    frame.TITLE = tk.Label(frame,text=title)
    frame.TITLE.pack(side="top")

def ADD_SELF_DESTRUCT(frame):
    frame.SELF_DESTRUCT = tk.Button(frame,text="REMOVE ANALYSIS",fg="red",command=frame.destroy)
    frame.SELF_DESTRUCT.pack(side="right")
    
def ADD_DATASET_NAME(frame,default_text=""):
    frame.DATASET_FRAME = tk.Frame(frame)
    frame.DATASET_FRAME.pack(side="top",fill="both")
    frame.dataset_label = tk.Label(frame.DATASET_FRAME,text="Dataset Name")
    frame.dataset_label.pack(side="left")
    frame.dataset_entry = tk.Entry(frame.DATASET_FRAME)
    frame.dataset_entry.insert(0, default_text)
    frame.dataset_entry.pack(side="right")

def FILE_OUTPUT_FRAME(frame,default_text=""):
    frame.FILE_OUTPUT=tk.Frame(frame)
    frame.FILE_OUTPUT.pack(side="top",fill='both')
    frame.file_label = tk.Label(frame.FILE_OUTPUT,text="Output filename")
    frame.file_label.pack(side="left")
    frame.file_entry = tk.Entry(frame.FILE_OUTPUT)
    frame.file_entry.insert(0, default_text)
    frame.file_entry.pack(side="right")

class SELECTION_MASK(tk.Frame):
    def __init__(self,master=None,label="Label",default_text=""):
        super().__init__(master)
        self.pack(side="top",fill='both')
        textlabel = tk.Label(self,text=label)
        textlabel.pack(side="left")
        self.selection_mask = tk.Entry(self)
        self.selection_mask.insert(0, default_text)
        self.selection_mask.pack(side="right")
    
class FileFrame(tk.Frame):
    def __init__(self,name,filetypes,master=None):
        super().__init__(master)
        self.filename = ""
        self.name     = name
        self.filetypes= filetypes
        self.pack(fill='both')
        self.button = tk.Button(self,text=self.name,command=self.GetFile)
        self.button.pack(side="left")
    def GetFile(self):
        self.filename = filedialog.askopenfilename(initialdir = os.getcwd(),
                                              title = "Select a File",
                                              filetypes = self.filetypes)
        self.button.configure(fg="green")
        tk.Label(self,text=self.filename).pack(side="right")
        
class RadioButtonBoolWithTextEntryFrame(tk.Frame):
    def __init__(self,name,cond_text,master=None):
        super().__init__(master)
        self.name     = name
        self.cond_text= cond_text
        self.entry = tk.Entry(self)
        self.custom_entry = ""
        self.last_val = ""
        self.custom_text = tk.Label(self,text=self.cond_text)
        self.values   = {"Yes":True,"No":False}
        self.boolean  = tk.BooleanVar(self,False)
        self.pack(fill='both')
        tk.Label(self, text=self.name).pack(side="left")
        for (text, value) in self.values.items():
            tk.Radiobutton(self, text = text, variable = self.boolean,
                value = value,command=self.update).pack(fill="x",side="right", ipady = 2,ipadx=2)
    def update(self):
        if self.boolean.get():
            self.entry.pack_forget()
            self.custom_text.pack_forget()
            self.entry.pack(side="right")
            self.custom_text.pack(side="right")
            self.entry.delete(10,tk.END)
            self.entry.insert(10,self.last_val)
        else:
            self.entry.pack_forget()
            self.custom_text.pack_forget()
            self.custom_entry = ""
    def get(self):
        self.custom_entry = self.entry.get()
        self.last_val     = self.entry.get()
        return self.boolean.get()

class CheckBoxWithField(tk.Frame):
    def __init__(self,master=None,prompt="",option="",entry_label=""):
        super().__init__(master)
        self.option = option
        self.prompt = prompt
        self.is_checked=tk.IntVar(self,0)
        self.pack(fill="both")
        self.CHECKBOX = tk.Checkbutton(self, text=self.prompt, variable=self.is_checked,command=self.button_checked)
        self.CHECKBOX.pack(side="left")
        self.spacing = tk.Label(self,text="\t")
        self.spacing.pack(side="left")
       
        self.LABEL = tk.Label(self,text=entry_label)
        self.TEXT = tk.Entry(self,state="disabled")
        
    def button_checked(self):
        if self.is_checked.get():
            self.TEXT.configure(state="normal")
            self.TEXT.pack(side="right")
            self.LABEL.pack(side="right")
        else:
            self.TEXT.configure(state="disabled")
            self.LABEL.pack_forget()
            self.TEXT.pack_forget()
        
    def command_string(self):
        if self.is_checked.get():
            return f"{self.option} {self.TEXT.get()} "
        return ""


    
class VerticalScrolledFrame: #https://gist.github.com/novel-yet-trivial/3eddfce704db3082e38c84664fc1fdf8
    """
    A vertically scrolled Frame that can be treated like any other Frame
    ie it needs a master and layout and it can be a master.
    :width:, :height:, :bg: are passed to the underlying Canvas
    :bg: and all other keyword arguments are passed to the inner Frame
    note that a widget layed out in this frame will have a self.master 3 layers deep,
    (outer Frame, Canvas, inner Frame) so 
    if you subclass this there is no built in way for the children to access it.
    You need to provide the controller separately.
    """
    def __init__(self, master, **kwargs):
        width = kwargs.pop('width', None)
        height = kwargs.pop('height', None)
        bg = kwargs.pop('bg', kwargs.pop('background', None))
        self.outer = tk.Frame(master, **kwargs)

        self.vsb = tk.Scrollbar(self.outer, orient=tk.VERTICAL)
        self.vsb.pack(fill=tk.Y, side=tk.RIGHT)
        self.canvas = tk.Canvas(self.outer, highlightthickness=0, width=width, height=height, bg=bg)
        self.canvas.pack(fill=tk.BOTH, expand=True)
        self.canvas['yscrollcommand'] = self.vsb.set
        # mouse scroll does not seem to work with just "bind"; You have
        # to use "bind_all". Therefore to use multiple windows you have
        # to bind_all in the current widget
        self.canvas.bind("<Enter>", self._bind_mouse)
        self.canvas.bind("<Leave>", self._unbind_mouse)
        self.vsb['command'] = self.canvas.yview

        self.inner = tk.Frame(self.canvas, bg=bg)
        # pack the inner Frame into the Canvas with the topleft corner 4 pixels offset
        self.canvas.create_window(4, 4, window=self.inner, anchor='nw')
        self.inner.bind("<Configure>", self._on_frame_configure)

        self.outer_attr = set(dir(tk.Widget))

    def __getattr__(self, item):
        if item in self.outer_attr:
            # geometry attributes etc (eg pack, destroy, tkraise) are passed on to self.outer
            return getattr(self.outer, item)
        else:
            # all other attributes (_w, children, etc) are passed to self.inner
            return getattr(self.inner, item)

    def _on_frame_configure(self, event=None):
        x1, y1, x2, y2 = self.canvas.bbox("all")
        height = self.canvas.winfo_height()
        self.canvas.config(scrollregion = (0,0, x2, max(y2, height)))

    def _bind_mouse(self, event=None):
        self.canvas.bind_all("<4>", self._on_mousewheel)
        self.canvas.bind_all("<5>", self._on_mousewheel)
        self.canvas.bind_all("<MouseWheel>", self._on_mousewheel)

    def _unbind_mouse(self, event=None):
        self.canvas.unbind_all("<4>")
        self.canvas.unbind_all("<5>")
        self.canvas.unbind_all("<MouseWheel>")

    def _on_mousewheel(self, event):
        """Linux uses event.num; Windows / Mac uses event.delta"""
        if event.num == 4 or event.delta > 0:
            self.canvas.yview_scroll(-1, "units" )
        elif event.num == 5 or event.delta < 0:
            self.canvas.yview_scroll(1, "units" )

    def __str__(self):
        return str(self.outer)


In [ ]:
### ANALYSIS FRAMES
### RMSD vs. Time
class RMSD(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"RMSD vs. Time")
        ### RMSD Selection Mask Frame
        self.RMSD_FRAME = SELECTION_MASK(self,"RMSD Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,default_text="RMSD.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        return f"rms first perres {self.RMSD_FRAME.selection_mask.get()} out {self.file_entry.get()}\n"

### Distance between two atoms vs. Time
class Distance(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Distance Analysis")
        ADD_DATASET_NAME(self,"d1")
        ### Atom Selections Frame
        self.ATOMS_FRAME = tk.Frame(self)
        self.ATOMS_FRAME.pack(side="top",fill='both')
        self.ATOM1_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 1 Selection Mask")
        self.ATOM2_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 2 Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,"Distances.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
        
    def command_string(self):
        dataset = self.dataset_entry.get()
        atom1 = self.ATOM1_FRAME.selection_mask.get()
        atom2 = self.ATOM2_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"distance {dataset} {atom1} {atom2} out {fileout}"

### Angle between three atoms vs. Time
class Angle(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Angle Analysis")
        ADD_DATASET_NAME(self,"a1")
        ### Atom Selections Frame
        self.ATOMS_FRAME = tk.Frame(self)
        self.ATOMS_FRAME.pack(side="top",fill='both')
        self.ATOM1_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 1 Selection Mask")
        self.ATOM2_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 2 Selection Mask")
        self.ATOM3_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 3 Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,"Angles.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
        
    def command_string(self):
        dataset = self.dataset_entry.get()
        atom1 = self.ATOM1_FRAME.selection_mask.get()
        atom2 = self.ATOM2_FRAME.selection_mask.get()
        atom3 = self.ATOM3_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"angle {dataset} {atom1} {atom2} {atom3} out {fileout}"
    
### Dihedral/Torsion between four atoms v. Time
class Torsion(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Dihedral/Torsion Analysis")
        ADD_DATASET_NAME(self,"t1")
        ### Atom Selections Frame
        self.ATOMS_FRAME = tk.Frame(self)
        self.ATOMS_FRAME.pack(side="top",fill='both')
        self.ATOM1_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 1 Selection Mask")
        self.ATOM2_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 2 Selection Mask")
        self.ATOM3_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 3 Selection Mask")
        self.ATOM4_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Atom 4 Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,"Dihedrals.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
        
    def command_string(self):
        dataset = self.dataset_entry.get()
        atom1 = self.ATOM1_FRAME.selection_mask.get()
        atom2 = self.ATOM2_FRAME.selection_mask.get()
        atom3 = self.ATOM3_FRAME.selection_mask.get()
        atom4 = self.ATOM4_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"dihedral {dataset} {atom1} {atom2} {atom3} {atom4} out {fileout}"
    
### Nucleotide Sugar Puckering v. Time
class NucleotidePucker(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Nucleotide Sugar Pucker Analysis")
        ADD_DATASET_NAME(self,"pucker1")
        ### Atom Selections Frame
        self.ATOMS_FRAME = tk.Frame(self)
        self.ATOMS_FRAME.pack(side="top",fill='both')
        self.ATOM1_FRAME = SELECTION_MASK(self.ATOMS_FRAME,"Residue Number")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,"NucleotidePucker.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
        
    def command_string(self):
        dataset = self.dataset_entry.get()
        res_num = self.ATOM1_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"pucker {dataset} :{res_num}@C1’ :{res_num}@C2’ :{res_num}@C3’ :{res_num}@C4’ :{res_num}@O4’ range360 out {fileout}\n"

### RMSF By Residue
class RMSF(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"RMSF By Residue")
        ADD_DATASET_NAME(self,"RMSF_ByRes")
        ### RMSF Selection Mask Frame
        self.RMSF_FRAME = SELECTION_MASK(self,"RMSF Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,default_text="RMSF_ByRes.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        dataset = self.dataset_entry.get()
        sel_mask = self.RMSF_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"atomicfluct {dataset} {sel_mask} out {fileout} byres"
        

### Correlated Motion between Residues
class CorrelatedMotion(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Correlated Motion")
        ADD_DATASET_NAME(self,"corrpath")
        ### RMSF Selection Mask Frame
        self.CORREL_FRAME = SELECTION_MASK(self,"Correlation Selection Mask")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,default_text="Correl.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        dataset = self.dataset_entry.get()
        sel_mask = self.CORREL_FRAME.selection_mask.get()
        fileout = self.file_entry.get()
        return f"matrix correl name {dataset} out {fileout} {sel_mask} byres"

### Normal Mode Analysis
class NormalModes(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Normal Modes Analysis")
        ADD_DATASET_NAME(self,"nma_data")
        ### RMSF Selection Mask Frame
        self.NMA_FRAME = SELECTION_MASK(self,"Normal Modes Selection Mask")
        self.NUM_MODES = SELECTION_MASK(self,"Number of Modes")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,default_text="normal_modes.nmd")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        dataset = self.dataset_entry.get()
        sel_mask = self.NMA_FRAME.selection_mask.get()
        num_modes = self.NUM_MODES.selection_mask.get()
        fileout = self.file_entry.get()
        com_str = f"matrix covar name {dataset} {sel_mask}\n"
        com_str+= f"diagmatrix {dataset} vecs {num_modes} reduce nmwiz mnwizvecs {num_modes} \\ \n"
        com_str+= f"nmwizfile {fileout} nmwizmask {sel_mask}\n"
        return com_str

### Hydrogen Bonding Analysis
class HydrogenBonds(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Hydrogen Bond Analysis")
        self.intramolecular = tk.BooleanVar(self,False)
        self.DISTANCE = SELECTION_MASK(self,"H-Bond cutoff distance",default_text="3.0")
        self.INTRAMOL = tk.Frame(self)
        self.label = tk.Label(self.INTRAMOL,text="Exclude intramolecular hydrogen bonds?")
        self.label.pack(side="left")
        for (text, value) in {"Yes":True,"No":False}.items():
            tk.Radiobutton(self.INTRAMOL, text = text, variable = self.intramolecular,
                value = value).pack(fill="x",side="right", ipady = 2,ipadx=2)
#         self.intramol = RadioButtonBoolWithTextEntryFrame(,"Selection Mask",master=self)
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,default_text="HBonds.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        nointramol = ""
        if self.intramolecular.get():
            nointramol = "nointramol"
        fileout = self.file_entry.get()
        distance = self.DISTANCE.selection_mask.get()
        return f"hbond dist {distance} avgout {fileout} {nointramol}"


### Radial Distribution Function
class RadialDistribution(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Radial Distribution Function")
        self.BINSPLIT = SELECTION_MASK(self,"Bin Separation",default_text="0.5")
        self.MAXDIST  = SELECTION_MASK(self,"Maximum Distance",default_text="10.0")
        self.SOLVENT  = SELECTION_MASK(self,"Solvent Molecule Mask",default_text=":WAT")
        ### File Output Frame
        FILE_OUTPUT_FRAME(self,default_text="RadialDistribution.dat")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
    def command_string(self):
        fileout = self.file_entry.get()
        split   = self.BINSPLIT.selection_mask.get()
        maxdist = self.MAXDIST.selection_mask.get()
        solvent = self.SOLVENT.selection_mask.get()
        return f"rdf out {fileout} {split} {maxdist} {solvent}"



In [ ]:
class SIEVE_TRAJECTORY_FRAME(tk.Frame):
    def __init__(self,master=None):
        super().__init__(master)
        self.pack()
        # Internal Variables
        self.bool_sieve = tk.BooleanVar(self,False)
        self.bool_random = tk.BooleanVar(self,False)
        
        # Internal subframes
        self.RADIO_FRAME = tk.Frame(self)
        self.ENTRY_FRAME = tk.Frame(self)
        self.USE_RANDOM_FRAMES = tk.Frame(self)
        self.RANDOM_SEED = tk.Frame(self)
        
        # Labels
        tk.Label(self.RADIO_FRAME, text = "Sieve Frames?").pack(side="left")
        tk.Label(self.USE_RANDOM_FRAMES, text = "Select Frames Randomly?").pack(side="left")
        self.ENTRY_PROMPT = tk.Label(self.ENTRY_FRAME,text="Stride")
        self.SEED_PROMPT  = tk.Label(self.RANDOM_SEED,text="seed")
        
        # Fields
        self.FIELD        = tk.Entry(self.ENTRY_FRAME)
        self.SEED_VALUE   = tk.Entry(self.RANDOM_SEED)
        
        # Packing and Positioning of Main Subframes
        self.RADIO_FRAME.pack(side="top")
        self.ENTRY_FRAME.pack(side="top")
        self.USE_RANDOM_FRAMES.pack(side="top")
        self.SEED_VALUE.pack(side="right")
        self.SEED_PROMPT.pack(side="right")

        # Radio Buttons
        for (text,value) in {"Yes":True,"No":False}.items():
            tk.Radiobutton(self.RADIO_FRAME, 
                           text     = text, 
                           variable = self.bool_sieve,
                           value    = value,
                           command  = self.update_radio_1).pack(fill="x",side="left",ipady = 2,ipadx=2)

        for (text,value) in {"Yes":True,"No":False}.items():
            tk.Radiobutton(self.USE_RANDOM_FRAMES, 
                           text     = text, 
                           variable = self.bool_random,
                           value    = value,
                           command  = self.update_radio_2).pack(fill="x",side="left",ipady = 2,ipadx=2)

        self.update_radio_1()
        self.update_radio_2()
        
    def update_radio_1(self):
        self.FIELD.pack_forget()
        self.ENTRY_PROMPT.pack_forget()
        self.USE_RANDOM_FRAMES.pack_forget()
        if self.bool_sieve.get():
            self.FIELD.pack(side="right")
            self.ENTRY_PROMPT.pack(side="right")
            self.USE_RANDOM_FRAMES.pack(side="top")
            self.FIELD.delete(0,tk.END)
            self.FIELD.insert(0,1)
    def update_radio_2(self):
        self.RANDOM_SEED.pack_forget()
        if self.bool_random.get():
            self.RANDOM_SEED.pack(side="top")
            self.SEED_VALUE.delete(0,tk.END)
            self.SEED_VALUE.insert(0,123456)
            
    def value(self):
        full_string = ""
        if self.bool_sieve.get():
            full_string += f"sieve {self.FIELD.get()} "
            if self.bool_random.get():
                full_string += "random "
                if self.SEED_VALUE.get() != "":
                    full_string += f"sieveseed {self.SEED_VALUE.get()} "
        return full_string


class Clust_Distance_Options(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="blue", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        TITLE_FRAME(self,"Clustering Distance Options")
        
        self.DISTANCE_SELECT = tk.Frame(self)
        self.DISTANCE_SELECT.pack(side="top")
        self.RADIO_1 = tk.Frame(self.DISTANCE_SELECT)
        self.RADIO_1.pack(side="top")
        
        ### FRAME SETS ###
        self.RMS_FRAME       = tk.Frame(self.DISTANCE_SELECT)
        self.RMS_FRAME.title = TITLE_FRAME(self.RMS_FRAME,"RMS Distance")
        self.RMS_FRAME.mask  = BOOLEAN_WITH_ENTRY(self.RMS_FRAME,"Use selection mask?","Selection Mask",":1-100","")
        self.RMS_FRAME.mass  = BOOLEAN_RESPONSE(self.RMS_FRAME,"Use Mass?","mass","")
        self.RMS_FRAME.nofit = BOOLEAN_RESPONSE(self.RMS_FRAME,"No Fit?","nofit","")
        
        self.SRMSD_FRAME       = tk.Frame(self.DISTANCE_SELECT)
        self.SRMSD_FRAME.title = TITLE_FRAME(self.SRMSD_FRAME,"SRMSD Distance")
        self.SRMSD_FRAME.mask  = BOOLEAN_WITH_ENTRY(self.SRMSD_FRAME,"Use selection mask?","Selection Mask",":1-100","")
        self.SRMSD_FRAME.mass  = BOOLEAN_RESPONSE(self.SRMSD_FRAME,"Use Mass?","mass","")
        self.SRMSD_FRAME.nofit = BOOLEAN_RESPONSE(self.SRMSD_FRAME,"No Fit?","nofit","")
        
        self.DME_FRAME       = tk.Frame(self.DISTANCE_SELECT)
        self.DME_FRAME.title = TITLE_FRAME(self.DME_FRAME,"DME Distance")
        self.DME_FRAME.mask  = ENTRY_FIELD(self.DME_FRAME,"Selection Mask",":1-100")
        
        self.DATA_FRAME       = tk.Frame(self.DISTANCE_SELECT)
        self.DATA_FRAME.title = TITLE_FRAME(self.DATA_FRAME,"Data Distance")
        self.DATA_FRAME.data  = ENTRY_FIELD(self.DATA_FRAME,"Dataset(s)","d1,a2,t3,...")
        
        self.radio_1 = tk.StringVar(self,"rms")
        for (value,text) in {"rms":"RMS","srmsd":"SRMSD","dme":"DME","data":"Data"}.items():
            tk.Radiobutton(self.RADIO_1, 
                           text     = text, 
                           variable = self.radio_1,
                           value    = value,
                           command  = self.update_radio_1).pack(fill="x",side="right",ipady = 2,ipadx=2)
            
        self.SIEVEFRAMES   = SIEVE_TRAJECTORY_FRAME(self)
        self.LOADPAIRDIST  = BOOLEAN_RESPONSE(self,"Load Pairwise Distance?","loadpairdist ","")
        self.SAVEPAIRDIST  = BOOLEAN_RESPONSE(self,"Save Pairwise Distance?","savepairdist ","")
        self.PAIRDISTFILE  = BOOLEAN_WITH_ENTRY(self,"Pairwise Distance Filename?","Filename","pairdist.dat",False)
        self.PAIRWISECACHE = RADIO_OPTIONS(self,"Pairwise Cache Option",{"Memory":"mem","Disk":"disk","None":"none"},"mem")
        self.INCLUDESIEVEINCALC = BOOLEAN_RESPONSE(self,"Include sieved frames in cluster average?","includesieveincalc ","")
        self.PWRECALC = BOOLEAN_RESPONSE(self,"Force Recalculation if pairwise file doesn't match?","pwrecalc ","")
        self.update_radio_1()
        
    def update_radio_1(self):
        self.RMS_FRAME.pack_forget()
        self.SRMSD_FRAME.pack_forget()
        self.DME_FRAME.pack_forget()
        self.DATA_FRAME.pack_forget()
        print(self.radio_1.get())
        if self.radio_1.get() == "rms":
            self.RMS_FRAME.pack(side="top")
        if self.radio_1.get() == "srmsd":
            self.SRMSD_FRAME.pack(side="top")
        if self.radio_1.get() == "dme":
            self.DME_FRAME.pack(side="top")
        if self.radio_1.get() == "data":
            self.DATA_FRAME.pack(side="top")
        
    def value(self):
        full_string = ""
        if self.radio_1.get() == "RMS":
            full_string += f"rms {self.RMS_FRAME.mask.value()} {self.RMS_FRAME.mass.value()} {self.RMS_FRAME.nofit.value()} "
        if self.radio_1.get() == "SRMSD":
            full_string += f"srmsd {self.SRMSD_FRAME.mask.value()} {self.SRMSD_FRAME.mass.value()} {self.SRMSD_FRAME.nofit.value()} "
        if self.radio_1.get() == "DME":
            full_string += f"dme {self.DME_FRAME.mask.value()} "
        if self.radio_1.get() == "Data":
            full_string += f"data {self.DATA_FRAME.data.value()} "
        full_string += f"{self.SIEVEFRAMES.value()}{self.LOADPAIRDIST.value()}{self.SAVEPAIRDIST.value()}pairwisecache {self.PAIRWISECACHE.value()}"
        if self.PAIRDISTFILE.value():
            full_string += f"pairdist {self.PAIRDISTFILE.value()}"
        full_string += f"{self.INCLUDESIEVEINCALC.value()}{self.PWRECALC.value()} "
        return full_string

# Output options:
# 	[out <cnumvtime>] [gracecolor] [summary <summaryfile>] [info <infofile>]
# 	[summarysplit <splitfile>] [splitframe <comma-separated frame list>]
#     [bestrep {cumulative|centroid|cumulative_nosieve}] [savenreps <#>]
#     [clustersvtime <filename> cvtwindow <window size>]
# 	[cpopvtime <file> [normpop | normframe] [lifetime]]
#     [sil <silhouette file prefix>] [assignrefs [refcut <rms>] [refmask <mask>]]
class Clust_Output_Options(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="blue", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Clustering Report Options")
        ### Options Frames

    def command_string(self):
        full_string  =  " "
        return full_string

# Coordinate output options:
#### CHECKBOXES



class Clust_Coordinate_Output_Options(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="blue", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        self.frames     = {}
        self.variables  = {}
        self.checkboxes = {}
        self.is_checked = {}
        self.entry_labels = {}
        
        ADD_TITLE(self,"Clustering Coordinate Output Options")
        ### Options Frames
        # clusterout <trajfileprefix> [clusterfmt <trajformat>] 
        key = "clusterout"
        self.frames[key] = tk.Frame(self)
        self.is_checked[key] = tk.IntVar(self,0)
        self.checkboxes[key] = tk.Checkbutton(self.frames[key], text=key+"\t", variable=self.is_checked[key])
        self.entry_labels[key] = tk.Label(self.frames[key],text="trajfileprefix")
        self.variables[key] = tk.Entry(self.frames[key])
        self.checkboxes[key].pack(side="left")
        self.variables[key].pack(side="right")
        self.entry_labels[key].pack(side="right")
        key = "cluserfmt"
        self.frames[key] = tk.Frame(self)
        self.is_checked[key] = tk.IntVar(self,0)
        self.checkboxes[key] = tk.Checkbutton(self.frames[key], text=key+"\t", variable=self.is_checked[key])
        self.entry_labels[key] = tk.Label(self.frames[key],text="trajformat")
        self.variables[key] = tk.Entry(self.frames[key])
        self.checkboxes[key].pack(side="left")
        self.variables[key].pack(side="right")
        self.entry_labels[key].pack(side="right")
        

        # singlerepout <trajfilename> [singlerepfmt <trajformat>]
        key = "singlerepout"
        self.frames[key] = tk.Frame(self)
        self.is_checked[key] = tk.IntVar(self,0)
        self.checkboxes[key] = tk.Checkbutton(self.frames[key], text=key+"\t", variable=self.is_checked[key])
        self.entry_labels[key] = tk.Label(self.frames[key],text="trajfilename")
        self.variables[key] = tk.Entry(self.frames[key])
        self.checkboxes[key].pack(side="left")
        self.variables[key].pack(side="right")
        self.entry_labels[key].pack(side="right")
        key = "singlerepfmt"
        self.frames[key] = tk.Frame(self)
        self.is_checked[key] = tk.IntVar(self,0)
        self.checkboxes[key] = tk.Checkbutton(self.frames[key], text=key+"\t", variable=self.is_checked[key])
        self.entry_labels[key] = tk.Label(self.frames[key],text="trajformat")
        self.variables[key] = tk.Entry(self.frames[key])
        self.checkboxes[key].pack(side="left")
        self.variables[key].pack(side="right")
        self.entry_labels[key].pack(side="right")
        
        # repout <repprefix> [repfmt <repfmt>] [repframe]
        key = "repout"
        self.frames[key] = tk.Frame(self)
        self.is_checked[key] = tk.IntVar(self,0)
        self.checkboxes[key] = tk.Checkbutton(self.frames[key], text=key+"\t", variable=self.is_checked[key])
        self.entry_labels[key] = tk.Label(self.frames[key],text="repprefix")
        self.variables[key] = tk.Entry(self.frames[key])
        self.checkboxes[key].pack(side="left")
        self.variables[key].pack(side="right")
        self.entry_labels[key].pack(side="right")
        key = "repfmt"
        self.frames[key] = tk.Frame(self)
        self.is_checked[key] = tk.IntVar(self,0)
        self.checkboxes[key] = tk.Checkbutton(self.frames[key], text=key+"\t", variable=self.is_checked[key])
        self.entry_labels[key] = tk.Label(self.frames[key],text="repfmt")
        self.variables[key] = tk.Entry(self.frames[key])
        self.checkboxes[key].pack(side="left")
        self.variables[key].pack(side="right")
        self.entry_labels[key].pack(side="right")

        # avgout <avgprefix> [avgfmt <avgfmt>] 
        key = "avgout"
        self.frames[key] = tk.Frame(self)
        self.is_checked[key] = tk.IntVar(self,0)
        self.checkboxes[key] = tk.Checkbutton(self.frames[key], text=key+"\t", variable=self.is_checked[key])
        self.entry_labels[key] = tk.Label(self.frames[key],text="avgprefix")
        self.variables[key] = tk.Entry(self.frames[key])
        self.checkboxes[key].pack(side="left")
        self.variables[key].pack(side="right")
        self.entry_labels[key].pack(side="right")
        key = "avgfmt"
        self.frames[key] = tk.Frame(self)
        self.is_checked[key] = tk.IntVar(self,0)
        self.checkboxes[key] = tk.Checkbutton(self.frames[key], text=key+"\t", variable=self.is_checked[key])
        self.entry_labels[key] = tk.Label(self.frames[key],text="avgfmt")
        self.variables[key] = tk.Entry(self.frames[key])
        self.checkboxes[key].pack(side="left")
        self.variables[key].pack(side="right")
        self.entry_labels[key].pack(side="right")

        for key in self.checkboxes.keys():
            self.frames[key].pack(side="top",fill="both")

    def command_string(self):
        full_string  =  ""
        for key in self.checkboxes.keys():
            if self.is_checked[key]:
                full_string += f"{key} {self.variables[key].get()} "
        return full_string

### MAIN CLUSTERING FRAME
class Clustering(tk.Frame):
    def __init__(self,master):
        super().__init__(master, highlightbackground="black", highlightthickness=2)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Clustering")
        self.options=["SELECT CLUSTERING ALGORITHM","Heirarchical Agglomerative", "DBSCAN", "Density Peaks","K-means"]
        self.algo_type = tk.StringVar(self,"SELECT CLUSTERING ALGORITHM")
        self.algo_type.set("SELECT CLUSTERING ALGORITHM")
        
### ADD THIS   # cluster↓ [crdset <crd set> | nocoords]
        
        
        ### Select Algorithm Frame
        self.ALGORITHM = tk.Frame(self, highlightbackground="blue", highlightthickness=2)
        self.ALGORITHM.pack()
        self.WORKING = tk.Frame(self.ALGORITHM)
        self.CURR_ALGO = tk.Frame(self.WORKING)
        self.CURR_ALGO.pack(side="top")
        self.algo_choice = tk.OptionMenu(self.ALGORITHM ,self.algo_type, *self.options, command=self.update_working)
        self.algo_choice.pack(side="top")
        
        ### Chosen Algorithm Frame
        self.WORKING.pack(side="top")
        self.DIST_OPTS = Clust_Distance_Options(self)
        self.OUTP_OPTS = Clust_Output_Options(self)
        self.COOR_OPTS = Clust_Coordinate_Output_Options(self)
        self.DIST_OPTS.pack(side="top")
        self.OUTP_OPTS.pack(side="top")
        self.COOR_OPTS.pack(side="top")
        ### SELF-DESTRUCT BUTTON
        ADD_SELF_DESTRUCT(self)
        
    def update_working(self,event):
        ## clear previous algorithm choice
        if self.algo_type.get() == "Heirarchical Agglomerative":
            self.CURR_ALGO = HeirAgglo(self.WORKING)
        elif self.algo_type.get() == "DBSCAN":
            self.CURR_ALGO = DBScan(self.WORKING)
        elif self.algo_type.get() == "Density Peaks":
            self.CURR_ALGO = DPeaks(self.WORKING)
        elif self.algo_type.get() == "K-means":
            self.CURR_ALGO = Kmeans(self.WORKING)
        else:
            self.CURR_ALGO = tk.Frame(self.WORKING)
        self.CURR_ALGO.pack(side="top")
        
    def command_string(self):
        full_string = self.CURR_ALGO.command_string()
        full_string += self.DIST_OPTS.command_string()
        full_string += self.OUTP_OPTS.command_string()
        full_string += self.COOR_OPTS.command_string()
        return full_string
        



In [ ]:
### HeirAgglo
# [hieragglo [epsilon <e>] [clusters <n>] [linkage|averagelinkage|complete]
#       [epsilonplot <file>] [includesieved_cdist]]
class HeirAgglo(tk.Frame):
    def __init__(self,master):
        super().__init__(master)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"HeirAgglo Clustering")
        ### Options Frames
        # Epsilon
        self.EPSILON     = SELECTION_MASK(self,"Epsilon",default_text="0.5")
        # Clusters
        self.CLUSTERS    = SELECTION_MASK(self,"Clusters",default_text="10")
        # linkage options
        self.LINKAGE_OPT = tk.Frame(self)
        self.LINKAGE_OPT.pack(side="top")
        self.linkage_opt = tk.StringVar(self,"linkage")
        self.LINKAGE_OPT.pack(fill='both')
        tk.Label(self.LINKAGE_OPT, text="").pack(side="left")
        for (text, value) in {"Linkage":"linkage","Average Linkage":"averagelinkage","Complete":"complete"}.items():
            tk.Radiobutton(self.LINKAGE_OPT, text = text, variable = self.linkage_opt,
                value = value).pack(fill="x",side="right", ipady = 2,ipadx=2)
        # Save Epsilon Plot        
        self.EPS_PLOT    = RadioButtonBoolWithTextEntryFrame("Save Epsilon Plot?","Filename",self)
        # Include Sieved C-dist
        self.INCL_SIEVED = tk.Frame(self)
        self.INCL_SIEVED.pack(side="top")
        self.includesieved = tk.BooleanVar(self,False)
        tk.Label(self.INCL_SIEVED, text="Include Sieved C-distance?").pack(side="left")
        for (text, value) in {"Yes":True,"No":False}.items():
            tk.Radiobutton(self.INCL_SIEVED, text = text, variable = self.includesieved,
                value = value).pack(fill="x",side="right", ipady = 2,ipadx=2)
    def command_string(self):
        full_string  =  "heiragglo "
        full_string += f"epsilon {self.EPSILON.selection_mask.get()} "
        full_string += f"clusters {self.CLUSTERS.selection_mask.get()} "
        full_string += f"{self.linkage_opt.get()} "
        if self.EPS_PLOT.get():
            full_string += f"epsilonplot {self.EPS_PLOT.custom_entry} "
        if self.includesieved.get():
            full_string += "includesieved_cdist "
        return full_string

In [ ]:
### DBScan
# [dbscan minpoints <n> epsilon <e> 
# [sievetoframe] [kdist <k> [kfile <prefix>]]]

class DBScan(tk.Frame):
    def __init__(self,master):
        super().__init__(master)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"DBSCAN Clustering")
        
        ### Variables
        self.minpoints = tk.IntVar(self,10)
        self.epsilon   = tk.DoubleVar(self,0.5)
        self.sievetoframe = tk.BooleanVar(self,False)
        self.kdist = tk.BooleanVar(self,False)
        self.kdistfile = tk.StringVar(self,"KDistPlot.dat")
        
        ### Options Frame
        
    def command_string(self):
        pass

In [ ]:
### DPeaks
# [dpeaks epsilon <e> [noise] [dvdfile <density_vs_dist_file>]
# 	  [choosepoints {manual | auto}]
# 	  [distancecut <distcut>] [densitycut <densitycut>]
# 	  [runavg <runavg_file>] [deltafile <file>] [gauss]]


class DPeaks(tk.Frame):
    def __init__(self,master):
        super().__init__(master)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"Density Peaks Clustering")
        
        ### Variables
        self.epsilon   = tk.DoubleVar(self,0.5)
        self.noise = tk.BooleanVar(self,False)
        self.dvd = tk.BooleanVar(self,False)
        self.dvdfile = tk.StringVar(self,"Density_vs_Distance.dat")
        self.choosepoints = tk.Stringvar(self,"auto")
        self.distancecut = tk.DoubleVar(self,5.0)
        self.densitycut = tk.DoubleVar(self,10.0)
        self.runavg =tk.BooleanVar(self,False)
        self.runavgfile = tk.StringVar(self,"Running_Average.dat")
        self.delta = tk.BooleanVar(self,False)
        self.deltafile = tk.StringVar(self,"Delta.dat")
        self.gauss = tk.BooleanVar(self,False)
        ### Options Frame
        
    def command_string(self):
        pass

In [ ]:
### K-means
# [kmeans clusters <n> [randompoint [kseed <seed>]] [maxit <iterations>]
#     [{readtxt|readinfo} infofile <file>]

class Kmeans(tk.Frame):
    def __init__(self,master):
        super().__init__(master)
        self.master = master
        self.pack(fill='both')
        ADD_TITLE(self,"K-means Clustering")
        
        ### Variables
        self.clusters   = tk.IntVar(self,10)
        self.randompoint = tk.BooleanVar(self,False)
        self.kseed = tk.BooleanVar(self,False)
        self.seed = tk.DoubleVar(self,1.0)
        self.maxit = tk.BooleanVar(self,False)
        self.iterations = tk.IntVar(self,1000)
        self.readtxt = tk.StringVar(self,"readinfo")
        self.infofile = tk.BooleanVar(self,False)
        self.infofilename = tk.StringVar(self,"infofile.dat")
        
        ### Options Frame
        
    def command_string(self):
        pass

In [ ]:
def reset_scrollregion(self, event):
    main_canvas.configure(scrollregion=main_canvas.bbox("all"))

### Main Analysis Frame
class Analysis_Frame(tk.Frame):
    def __init__(self,master=None):
        super().__init__(master)
        self.master = master
        self.options=["SELECT ANALYSIS","RMSD","RMSF","Correlated Motion","Normal Modes",
                      "Hydrogen Bond Analysis","Radial Distribution Function",
                      "Distance","Angle","Dihedral/Torsion","Clustering"]
        self.analysis_type = tk.StringVar()
        self.analysis_type.set("SELECT ANALYSIS")
        self.pack(fill="both")

        ### ADD NEW ANALYSIS FRAME
        self.ADD_ANALYSIS_FRAME = tk.Frame(self)
        self.ADD_ANALYSIS_FRAME.pack(side="bottom")
        self.drop = tk.OptionMenu(self.ADD_ANALYSIS_FRAME ,self.analysis_type, *self.options)
        self.drop.pack(side="left")
        self.button = tk.Button(self.ADD_ANALYSIS_FRAME,text="ADD ANALYSIS",command=self.add_analysis)
        self.button.pack(side="left")

    def add_analysis(self):
        antype = self.analysis_type.get()
        self.ADD_ANALYSIS_FRAME.pack_forget()
        if antype == "RMSD":
            newframe = RMSD(self)
        if antype == "RMSF":
            newframe = RMSF(self)
        if antype == "Correlated Motion":
            newframe = CorrelatedMotion(self)
        if antype == "Normal Modes":
            newframe = NormalModes(self)
        if antype == "Hydrogen Bond Analysis":
            newframe = HydrogenBonds(self)
        if antype == "Radial Distribution Function":
            newframe = RadialDistribution(self)
        if antype == "Distance":
            newframe = Distance(self)
        if antype == "Angle":
            newframe = Angle(self)
        if antype == "Dihedral/Torsion":
            newframe = Torsion(self)
        if antype == "Clustering":
            newframe = Clustering(self)
        if antype != "SELECT ANALYSIS":
            newframe.pack(side="top")
        self.ADD_ANALYSIS_FRAME.pack(side="bottom")
        reset_scrollregion(root, "")
    def command_string(self):
        return_string = ""
        for frame in self.pack_slaves():
            if getattr(frame,"command_string",False):
                return_string += frame.command_string()
                return_string += "\n"
        return_string += "\n"
        return return_string
        

In [ ]:
class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        ### INTERNAL VARIABLES ###
        self.BooleanOptions = ["Autoimage","StripWater","StripIons"]
        self.master = master
        self.trajectory_file = ""
        self.prmtop_file     = ""
        self.autoimage_bool  = tk.BooleanVar(self,False)
        self.stripwater_bool = tk.BooleanVar(self,False)
        self.stripions_bool  = tk.BooleanVar(self,False)
        self.rmsf_sel_mask   = ""
        self.rmsd_sel_mask   = ""
        self.cations         = "K+"
        self.anions          = "Cl-"

        ### FRAMES ###
        ## NON TEXT SIDE
        self.NONTEXT = tk.Frame(self)
        self.NONTEXT.pack(side="left",fill="both")
        ## TEXT SIDE
        self.TEXT = tk.Frame(self)
        self.TEXT.pack(side="right",fill="both")
        self.display_commands_frame = tk.Frame(self.TEXT)
        self.frame_run_or_quit = tk.Frame(self.TEXT)

        ### FINAL INITIALIZATION OF PROGRAM ###
        self.pack()
        self.create_widgets()

    def create_widgets(self):
        ### FileFrames
        self.prmtopfileframe = FileFrame("Prmtop File",(("AMBER Parm7","*.prmtop*"),("all files","*.*")),master=self.NONTEXT)
        self.trajectoryfileframe  = FileFrame("Trajectory File",(("CHARMM DCD","*.dcd*"),("AMBER MDCRD","*.mdcrd*"),("all files","*.*")),master=self.NONTEXT)

        ### RadioButton Options
        self.frame_autoimage = RadioButtonBoolWithTextEntryFrame("Autoimage?","Selection Mask",master=self.NONTEXT)
        self.frame_stripwater = RadioButtonBoolWithTextEntryFrame("Strip solvent?","Solvent Mask",master=self.NONTEXT)
        self.frame_stripions = RadioButtonBoolWithTextEntryFrame("Strip counterions?","Counterions Mask",master=self.NONTEXT)

        # Add Analyses
        self.ANALYSES = Analysis_Frame(self.NONTEXT)
        self.ANALYSES.pack(fill="both")
        
        # DISPLAY CURRENT COMMANDS
        self.display_commands_frame.pack(side="top",fill="both")
        self.T = tk.Text(self.display_commands_frame, height=40, width=120)
        self.commands_list="this is a test of commands\nsecondline\n now a third line\n  four score and seven whatever"
        self.T.pack()
        
        # RUN CPPTRAJ or QUIT Frame
        self.frame_run_or_quit.pack(side="bottom",fill="both")
        self.quit = tk.Button(self.frame_run_or_quit,text="QUIT",fg="red",command=root.destroy)
        self.quit.pack(side="right")
        self.run_cpptraj = tk.Button(self.frame_run_or_quit,text="RUN CPPTRAJ",fg="green",command=self.run_cpptraj)
        self.run_cpptraj.pack(side="right")
        self.update_text = tk.Button(self.frame_run_or_quit,text="UPDATE_TEXT",fg="blue",command=self.update_Text)
        self.update_text.pack(side="right")
        
        
    def run_cpptraj(self):
        self.update_Text()
        with open("cpptraj.in","w") as f:
            f.write(self.commands_list)
        f.close()
        subprocess.call("cpptraj < cpptraj.in > cpptraj.out", shell=True)
        # generate cpptraj.in based on all variables, then call cpptraj.

    
    def update_Text(self):
        self.commands_list  = f"parm {self.prmtopfileframe.filename}\n"
        self.commands_list += f"trajin {self.trajectoryfileframe.filename}\n"
        if self.frame_autoimage.get():
            self.commands_list+=f"autoimage anchor {self.frame_autoimage.custom_entry}\n"
        if self.frame_stripwater.get():
            self.commands_list+=f"strip {self.frame_stripwater.custom_entry}\n"
        if self.frame_stripions.get():
            self.commands_list+=f"strip {self.frame_stripions.custom_entry}\n"
        ### Add Analyses
#         print(self.ANALYSES.pack_slaves())
        self.commands_list += self.ANALYSES.command_string()
        self.commands_list += "run\n"
        
        ### UPDATE DISPLAYED TEXT ###    
        self.T.delete(1.0,tk.END)
        self.T.insert(tk.END,self.commands_list) 
        self.T.config(height=len(self.commands_list.split("\n"))+1)
        self.T.pack()








In [ ]:
def _on_mousewheel(self, event):
    self.canvas.yview_scroll(-1*(event.delta/120), "units")


### RUN MAIN APPLICATION
root = tk.Tk()
root.title("CPPTRAJ Analysis Generator")
root.geometry("1600x900")

#Scrollable Main Window
main_frame = tk.Frame(root)
main_frame.pack(fill="both",expand=1)
scroll = tk.Frame(main_frame)
scroll.pack(fill="y",side="right")
main_canvas= tk.Canvas(main_frame)
main_canvas.pack(side="left",fill="both",expand=1)
y_scrollbar = tk.Scrollbar(main_frame,orient="vertical",command=main_canvas.yview)
y_scrollbar.pack(side="right",fill="y")
main_canvas.configure(yscrollcommand=y_scrollbar.set)
root.bind("<Configure>",lambda e: main_canvas.config(scrollregion= main_canvas.bbox("all"))) 
root.bind("<FocusOut>",lambda e: main_canvas.config(scrollregion= main_canvas.bbox("all"))) 
root.bind("<FocusIn>",lambda e: main_canvas.config(scrollregion= main_canvas.bbox("all"))) 
root.bind_all("<MouseWheel>", _on_mousewheel)



submain = tk.Frame(main_canvas)
main_canvas.create_window((0,0), window=submain, anchor="nw")




app = Application(submain)


root.mainloop()